In [1]:
from glob import glob
import numpy as np
import os
os.chdir('/data/hyeryung/mucoco')
import pandas as pd
# from evaluation.prompted_sampling.evaluate import distinctness, repetition
from transformers import AutoTokenizer
from tqdm import tqdm

In [2]:
tokenizer = AutoTokenizer.from_pretrained('gpt2-large')

In [20]:
data = []
for fpath in sorted(glob('/data/hyeryung/BOLT/sentiment/sentiment/*.jsonl')):
    output = pd.read_json(fpath, lines=True)
    output['fpath'] = fpath
    data.append(output)
data = pd.concat(data, axis=0)

In [21]:
data = data.explode('generations')

In [22]:
data['seq_len'] = data['generations'].apply(lambda x: len(tokenizer.encode(x['text'])))

In [23]:
data.seq_len.sum()

49196

In [24]:
all_data = []
for fpath in sorted(glob('/data/hyeryung/BOLT/sentiment/sentiment/*.sbertscore')):
    with open(fpath , 'r') as f:
        raw_data = f.readlines()
        tmp_data = [float(x.strip()) for x in raw_data[1:]]
    all_data.extend(tmp_data)
data['sbert']=all_data
print(np.mean(all_data))


0.029692789159978725


In [25]:
all_data = []
for fpath in sorted(glob('/data/hyeryung/BOLT/sentiment/sentiment/*.fluency')):
    with open(fpath , 'r') as f:
        tmp_data = [1 if x.strip() == 'LABEL_1' else 0 for x in f.readlines()]
    all_data.extend(tmp_data)
data['fluency'] = all_data
print(np.mean(all_data))

0.9566666666666667


In [26]:
all_data = []
for fpath in sorted(glob('/data/hyeryung/BOLT/sentiment/sentiment/*.ppl-big')):
    with open(fpath , 'r') as f:
        tmp_data = [float(x.strip().split(',')[0]) for x in f.readlines()]
    all_data.extend(tmp_data)
data['ppl'] = all_data
print(np.mean(all_data))

8.614294544184443


In [27]:
all_data = []
for fpath in sorted(glob('/data/hyeryung/BOLT/sentiment/sentiment/*.repetitions')):
    with open(fpath , 'r') as f:
        tmp_data = [0 if x.strip() == "{}" else 1 for x in f.readlines()]
    all_data.extend(tmp_data)
data['rep']=all_data
print(np.mean(all_data))


0.0


In [28]:
all_data = []
for fpath in sorted(glob('/data/hyeryung/BOLT/sentiment/sentiment/pos*.sentiment_ext')):
    
    with open(fpath , 'r') as f:
        raw_data = f.readlines()
        tmp_data = [1 if eval(x)['label']=='POSITIVE' else 0 for x in raw_data]
    all_data.extend(tmp_data)
    
for fpath in sorted(glob('/data/hyeryung/BOLT/sentiment/sentiment/neg*.sentiment_ext')):
    
    with open(fpath , 'r') as f:
        raw_data = f.readlines()
        tmp_data = [1 if eval(x)['label']=='NEGATIVE' else 0 for x in raw_data]
    all_data.extend(tmp_data)
    
data['sentiment_acc'] = all_data
print(np.mean(all_data))


0.7616666666666667


In [1]:
# ## dist-3

dist3_metrics=[]
for fpath in sorted(glob('/data/hyeryung/BOLT/sentiment/sentiment/*.jsonl')):

    outputs=pd.read_json(fpath, lines=True)
    _,_,dist3=distinctness(outputs)
    dist3_metrics.append(dist3)
#
print(np.mean(dist3_metrics))

NameError: name 'glob' is not defined

In [32]:
data['target']=data['fpath'].str.split('/').apply(lambda x: x[-1][:3])

In [37]:
data.loc[data['sbert']>= 0.5,].shape[0]/data.shape[0]

0.0005555555555555556

In [39]:
data.loc[data['sbert']>= 0.5,]

,prompt,generations,fpath,seq_len,sbert,fluency,ppl,rep,sentiment_acc,target
1,{'text': 'The book'},{'text': ' is full of interesting insights and...,/data/hyeryung/BOLT/sentiment/sentiment/pos.le...,12,0.537997,1,11.719075,0,0,pos


In [40]:
data.loc[data['sbert']< 0.5,].shape[0]

1799

In [41]:
data.loc[data['sbert']>= 0.5, ['sbert','fluency','ppl','rep','sentiment_acc']].mean()

sbert             0.537997
fluency           1.000000
ppl              11.719075
rep               0.000000
sentiment_acc     0.000000
dtype: float64

In [35]:
data.loc[data['sbert']< 0.5, ['fluency','ppl','rep','sentiment_acc']].mean()

fluency          0.956643
ppl              8.612569
rep              0.000000
sentiment_acc    0.762090
dtype: float64

In [13]:
fpath_prefix='/data/hyeryung/BOLT/detoxic/detoxic/'

In [14]:
data = []
for fpath in sorted(glob(f'{fpath_prefix}*.jsonl')):
    output = pd.read_json(fpath, lines=True)
    output['fpath'] = fpath
    data.append(output)
data = pd.concat(data, axis=0)

In [15]:
data = data.explode('generations')

In [16]:
data['seq_len'] = data['generations'].apply(lambda x: len(tokenizer.encode(x['text'])))

In [17]:
data.seq_len.sum()

93152

In [18]:
fpath

'/data/hyeryung/BOLT/detoxic/detoxic/gen_len20.jsonl'

In [20]:
all_data = []
for fpath in sorted(glob(f'{fpath_prefix}/*.sbertscore')):
    with open(fpath , 'r') as f:
        raw_data = f.readlines()
        tmp_data = [float(x.strip()) for x in raw_data[1:]]
    all_data.extend(tmp_data)
data['sbert']=all_data
print(np.mean(all_data))


0.031783755126808685


In [23]:
all_data = []
for fpath in sorted(glob(f'{fpath_prefix}/*.fluency')):
    with open(fpath , 'r') as f:
        tmp_data = [1 if x.strip() == 'LABEL_1' else 0 for x in f.readlines()]
    all_data.extend(tmp_data)
data['fluency'] = all_data
print(np.mean(all_data))

0.9592


In [24]:
all_data = []
for fpath in sorted(glob(f'{fpath_prefix}/*.ppl-big')):
    with open(fpath , 'r') as f:
        tmp_data = [float(x.strip().split(',')[0]) for x in f.readlines()]
    all_data.extend(tmp_data)
data['ppl'] = all_data
print(np.mean(all_data))

8.89616156043901


In [25]:
all_data = []
for fpath in sorted(glob(f'{fpath_prefix}/*.repetitions')):
    with open(fpath , 'r') as f:
        tmp_data = [0 if x.strip() == "{}" else 1 for x in f.readlines()]
    all_data.extend(tmp_data)
data['rep']=all_data
print(np.mean(all_data))


0.0008


In [27]:
all_data = []
for fpath in sorted(glob(f'{fpath_prefix}/*.toxicity')):
    raw_data = pd.read_json(fpath, lines=True)
    tmp_data = unravel_toxicity_data(raw_data)
    all_data.extend(tmp_data['toxicity'])
    
    
data['toxicity'] = all_data
print(np.mean(all_data))

0.08984012971584


In [38]:
data['prompt'] = data['prompt'].apply(lambda x: x['text'])

In [39]:
data.groupby('prompt')['toxicity'].max().mean()

0.21490436696

In [41]:
(data.groupby('prompt')['is_toxic'].sum() >= 1).mean()

0.06

In [30]:
data['sbert_geq_50']=data['sbert'] >= 0.5

In [31]:
data['is_toxic'] = data['toxicity'] >= 0.5

In [33]:
data.groupby('sbert_geq_50')[['toxicity', 'is_toxic','ppl', 'fluency','rep', 'sbert',
       ]].mean().sort_index(ascending=False)

,toxicity,is_toxic,ppl,fluency,rep,sbert
sbert_geq_50,,,,,,
True,0.214121,0.250000,4.280235,1.000000,0.000000,0.535930
False,0.089441,0.012039,8.910980,0.959069,0.000803,0.030165


In [34]:
data.groupby('sbert_geq_50').size().sort_index(ascending=False)

sbert_geq_50
True        8
False    2492
dtype: int64

In [35]:
data.groupby('sbert_geq_50').size().sort_index(ascending=False)/data.shape[0]

sbert_geq_50
True     0.0032
False    0.9968
dtype: float64